In [3]:
%pip install xgboost


   ---------------------------------------- 0.0/124.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/124.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/124.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/124.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/124.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/124.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/124.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/124.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/124.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/124.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/124.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/124.9 MB ? eta -:--:--
   ---------------------------------------- 0.3/124.9 MB ? eta -:--:--
   ---------------------------------------- 0.3/124.9 MB ? eta -:--:--
   --

In [6]:
import pandas as pd
import re
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from imblearn.over_sampling import SMOTE
from collections import Counter
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
import csv

# 1. Đọc dữ liệu từ file CSV
data = pd.read_csv('./dataset/Amazon_Product_Review_cleaned.csv')

# Hiển thị các cột của dataset (để kiểm tra)
print("Các cột trong dataset:", data.columns.tolist())

# 2. Tiền xử lý văn bản trong cột 'review'
def preprocess_text(text):
    text = str(text)            # Đảm bảo text là chuỗi
    text = text.lower()         # Chuyển về chữ thường
    # Giữ lại các ký tự chữ cái (bao gồm có dấu), số và dấu cách
    text = re.sub(r'[^a-zA-Z0-9À-ỹ\s]', '', text)
    return text

# Áp dụng tiền xử lý cho cột review
data['review'] = data['review_headline'].apply(preprocess_text)

# 3. Loại bỏ các dòng có giá trị bị thiếu
data = data.dropna(subset=['review', 'star_rating'])

# 4. Đảm bảo cột star_rating ở dạng số nguyên và bắt đầu từ 0
data['star_rating'] = data['star_rating'].astype(int) - 1

# Hiển thị các giá trị star_rating độc nhất
print("Các star_rating độc nhất:", data['star_rating'].unique())

from xgboost import XGBClassifier

# 1. Tách dữ liệu thành tập huấn luyện và tập kiểm tra
X = data['review']
y = data['star_rating']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 2. Chuyển đổi văn bản thành các đặc trưng TF-IDF
tfidf_vectorizer = TfidfVectorizer(max_features=5000)
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

# 3. Huấn luyện mô hình XGBoost
xgb_model = XGBClassifier(use_label_encoder=False, eval_metric='mlogloss')
xgb_model.fit(X_train_tfidf, y_train)

# 4. Dự đoán
y_pred_xgb = xgb_model.predict(X_test_tfidf)

# 5. Đánh giá
print("🔹 Accuracy (XGBoost):", accuracy_score(y_test, y_pred_xgb))
print("🔹 Classification Report:\n", classification_report(y_test, y_pred_xgb))
print("🔹 Confusion Matrix:\n", confusion_matrix(y_test, y_pred_xgb))


Các cột trong dataset: ['marketplace', 'customer_id', 'review_id', 'product_id', 'product_parent', 'product_title', 'product_category', 'star_rating', 'helpful_votes', 'total_votes', 'vine', 'verified_purchase', 'review_headline', 'review_body', 'review_date', 'sentiment']
Các star_rating độc nhất: [4 0 2 3 1]


c:\Users\Thinh.LAPTOP-GCU8T0AJ\.conda\envs\lugak1\lib\site-packages\xgboost\core.py:158: UserWarning: [11:01:49] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-08cbc0333d8d4aae1-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


🔹 Accuracy (XGBoost): 0.7646677471636953
🔹 Classification Report:
               precision    recall  f1-score   support

           0       0.58      0.43      0.50       344
           1       0.63      0.26      0.37       237
           2       0.63      0.33      0.44       462
           3       0.74      0.43      0.54      1159
           4       0.79      0.97      0.87      3968

    accuracy                           0.76      6170
   macro avg       0.67      0.49      0.54      6170
weighted avg       0.75      0.76      0.74      6170

🔹 Confusion Matrix:
 [[ 149    9   22   21  143]
 [  39   62   16   21   99]
 [  36   16  154   55  201]
 [  19    7   37  500  596]
 [  13    5   14   83 3853]]
